In [13]:
import pickle
from urllib.parse import urlparse
from fuzzywuzzy import fuzz
import pandas as pd
with open('company_url_dict_updated.pkl', 'rb') as f:
    student_careers_url = pickle.load(f)
df = pd.read_csv('companies_wikipedia_data.csv')
df['website'] = df['website'].fillna('')
df['student_careers_url'] = df['student_careers_url'].fillna('')
count = 0
removal_list = []
new_website_urls_for_lesser_known_companies = {}

for i, r in df.iterrows():
    parsed_uri = urlparse(student_careers_url[r['index_name']])
    if r['website'] is not '':
        parsed_uri = urlparse(r['website'])
    if parsed_uri is '':
        removal_list.append(i)
        continue
    hostname = '{uri.scheme}://{uri.netloc}/'.format(uri=parsed_uri)
    match_index = fuzz.token_set_ratio(r['name'].lower(), hostname)
    if match_index > 90:
        count+=1
        new_website_urls_for_lesser_known_companies[i] = hostname
    else:
        # abc xyz pqr -> (axp.com)
        removal_list.append(i)

print('Companies available:', count)
print('Companies removed:', len(removal_list))
for i in new_website_urls_for_lesser_known_companies.keys():
    df.at[i, 'website'] = new_website_urls_for_lesser_known_companies[i]
df = df.drop(removal_list)
print('Unique websites: ', len(df['website'].unique()))
df = df[['name','industry','type','founded','num_employees','revenue','operating_income','net_income','student_careers_url', 'website']]
# df.to_csv('temp_companies_wikipedia_data.csv', index=False)

Companies available: 289
Companies removed: 1761
Unique websites:  287


In [14]:
d1 = pd.read_csv('d1.csv')
url_domain_page_rank = {}
for i, r in d1.iterrows():
    try:
        url_domain_page_rank[r['Domain']] = float(str(r['Page Rank Value']))
        print(r['Domain'], float(str(r['Page Rank Value'])))
    except:
        continue
list_page_ranks = []
for i, r in df.iterrows():
    domain = urlparse(r['website']).netloc
    if 'www.' in domain:
        domain = domain[4:]
    if domain in url_domain_page_rank.keys():
        list_page_ranks.append(url_domain_page_rank[domain])
    else:
        list_page_ranks.append(0.0)
df['page_rank'] = list_page_ranks
df.to_csv('temp_companies_wikipedia_data.csv')
df.columns

247.ai 5.68
angel.co 6.87
33across.com 5.68
3dsystems.com 5.85
3alitytechnica.com 4.34
us.4d.com 4.53
4info.com 5.28
a10networks.com 5.38
aciworldwide.com 5.42
adaranetworks.com 4.57
apc.com 5.94
arcos-inc.com 4.72
asg.com 5.37
att.com 6.65
elemental.com 5.19
abacusnext.com 5.62
abbott.com 6.12
accordent.com 5.2
accusystem.com 4.19
boards.greenhouse.io 5.97
accusoft.com 6.16
acorns.com 6.16
adobe.com 7.88
actian.com 5.48
actifio.com 5.46
epicor.com 5.44
activenetwork.com 5.66
opentext.com 6.04
acumatica.com 5.69
acxiom.com 5.78
google.com 10.0
careers.microsoft.com 5.94
adaptiveplanning.com 4.9
addepar.com 5.59
adeptia.com 5.04
aditi.com 4.8
quantum.com 5.62
adventhealth.com 5.63
quest.com 5.68
aestiva.com 4.02
afiniti.com 5.13
agiloft.com 5.12
agilysys.com 5.0
aha.io 5.92
silive.com 6.15
airbiquity.com 5.49
descartes.com 5.49
airship.com 5.47
mckinsey.com 7.01
rocketsoftware.com 5.66
alfresco.com 5.72
alientechnology.com 5.25
allegrodev.com 4.68
allscripts.com 5.51
alphaeon.com 4.56
a

Index(['name', 'industry', 'type', 'founded', 'num_employees', 'revenue',
       'operating_income', 'net_income', 'student_careers_url', 'website',
       'page_rank'],
      dtype='object')